# 🚀 LangChain Agents Workshop: From Simple to Advanced

Welcome to the LangChain Agents Workshop! This hands-on tutorial will take you on an exciting journey from basic agent concepts to advanced multi-agent systems.

## 🎯 What You'll Build

1. **🧬 Genetic Agent** - Start simple with a basic conversational agent
2. **☁️ Azure AI Foundry Agent** - Level up with cloud-powered AI capabilities  
3. **💬 Group Chat System** - Master advanced multi-agent orchestration

## 🛠️ Quick Setup

This section sets up everything you need to run the agents in this notebook with minimal friction.

**What it does:**
- Installs Python dependencies and the shared local library
- Lets you provide API keys (Azure AI Inference and optional providers)
- Saves them to a local .env for reuse (optional)
- Verifies the project structure
- Optionally runs a tiny smoke test if keys are present

**Instructions:** Proceed top-to-bottom; each step is self-checking and safe to rerun.

In [ ]:
# 🔧 Step 1: Install Dependencies
# Safe to rerun multiple times

import os, sys, subprocess, pathlib

nb_dir = pathlib.Path().resolve()
project_root = nb_dir.parents[2] if (len(nb_dir.parents) >= 2) else nb_dir
lc_dir = nb_dir  # this notebook lives in Backend/python/langchain
shared_dir = lc_dir.parent / "shared"
req_file = lc_dir / "requirements.txt"

print(f"🔍 Notebook dir: {nb_dir}")
print(f"🏠 Project root: {project_root}")
print(f"📦 Using requirements: {req_file}")
print(f"🔗 Shared package dir: {shared_dir}")

def run_command(cmd):
    print(f"\n💻 Running: {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if result.returncode != 0:
        print(f"❌ Error: {result.stderr}")
        raise SystemExit(f"Command failed with exit code {result.returncode}")
    if result.stdout:
        print(f"✅ Output: {result.stdout}")

# Install dependencies using pip magic commands (keeps kernel environment clean)
try:
    import IPython
    get_ipython  # Verify we're in IPython/Jupyter
    
    print("\n📥 Installing requirements...")
    if req_file.exists():
        get_ipython().run_line_magic("pip", f"install -r {req_file}")
        print("✅ Requirements installed!")
    else:
        print("⚠️ requirements.txt not found; skipping dependency install.")
    
    print("\n🔗 Installing shared library...")
    if (shared_dir / "setup.py").exists():
        get_ipython().run_line_magic("pip", f"install -e {shared_dir}")
        print("✅ Shared library installed!")
    else:
        print("⚠️ Shared library setup.py not found; skipping -e install.")
        
except Exception as e:
    print(f"⚠️ IPython magic not available, falling back to subprocess: {e}")
    if req_file.exists():
        run_command(f"python -m pip install -r \"{req_file}\"")
    if (shared_dir / "setup.py").exists():
        run_command(f"python -m pip install -e \"{shared_dir}\"")

print("\n🎉 Dependencies installation complete!")

In [ ]:
# 🔑 Step 2: Configure API Keys
# Secure setup for Azure AI and other providers

import os
from dotenv import load_dotenv, set_key
from pathlib import Path

# Load existing environment
load_dotenv()

def get_or_prompt(env_var, description, required=True):
    """Get environment variable or prompt user for input."""
    value = os.getenv(env_var)
    if value:
        print(f"✅ {env_var} already configured")
        return value
    
    if required:
        print(f"\n🔑 Please provide your {description}:")
        value = input(f"{env_var}: ").strip()
        if not value and required:
            raise ValueError(f"{env_var} is required!")
    else:
        print(f"\n🔑 Optional: {description} (press Enter to skip):")
        value = input(f"{env_var}: ").strip()
    
    return value if value else None

def save_to_env_file(env_vars):
    """Save environment variables to .env file."""
    env_file = Path(".env")
    
    print(f"\n💾 Saving configuration to {env_file}...")
    for key, value in env_vars.items():
        if value:
            set_key(env_file, key, value)
            os.environ[key] = value
            print(f"✅ Saved {key}")

# Configure required and optional API keys
print("🔧 Setting up API configuration...")
print("📝 Note: API keys will be saved to .env file for future use")

env_vars = {}

# Required: Azure AI Inference
env_vars["AZURE_AI_INFERENCE_ENDPOINT"] = get_or_prompt(
    "AZURE_AI_INFERENCE_ENDPOINT", 
    "Azure AI Inference Endpoint (e.g., https://models.inference.ai.azure.com)"
)
env_vars["AZURE_AI_INFERENCE_API_KEY"] = get_or_prompt(
    "AZURE_AI_INFERENCE_API_KEY", 
    "Azure AI Inference API Key"
)

# Optional: Additional providers
env_vars["OPENAI_API_KEY"] = get_or_prompt(
    "OPENAI_API_KEY", 
    "OpenAI API Key (optional)", 
    required=False
)

env_vars["ANTHROPIC_API_KEY"] = get_or_prompt(
    "ANTHROPIC_API_KEY", 
    "Anthropic API Key (optional)", 
    required=False
)

# Save configuration
save_to_env_file(env_vars)

print("\n🎉 API configuration complete!")
print("🔄 Environment variables loaded and ready to use")

In [ ]:
# 🔍 Step 3: Verify Project Structure
# Ensure all required components are in place

import sys
from pathlib import Path

def check_structure():
    """Verify the project has the expected structure."""
    print("🔍 Verifying project structure...")
    
    # Add shared modules to path
    shared_path = str(nb_dir.parent / "shared")
    if shared_path not in sys.path:
        sys.path.insert(0, shared_path)
        print(f"✅ Added shared path: {shared_path}")
    
    # Check key directories and files
    checks = [
        (shared_dir, "Shared library directory"),
        (shared_dir / "shared", "Shared package"),
        (lc_dir / "agents", "LangChain agents directory"),
        (lc_dir / "config.yml", "Configuration file"),
        (req_file, "Requirements file")
    ]
    
    all_good = True
    for path, description in checks:
        if path.exists():
            print(f"✅ {description}: {path}")
        else:
            print(f"❌ Missing {description}: {path}")
            all_good = False
    
    return all_good

# Verify structure
structure_ok = check_structure()

if structure_ok:
    print("\n🎉 Project structure verification complete!")
    print("🚀 Ready to start building agents!")
else:
    print("\n⚠️ Some project components are missing.")
    print("Please ensure you're running this notebook from the correct directory.")

In [ ]:
# 🧪 Step 4: Quick Smoke Test
# Test basic connectivity and functionality

print("🧪 Running quick smoke test...")

try:
    # Test imports
    print("📦 Testing imports...")
    from shared import AgentRegistry, AgentConfig, AgentMessage
    print("✅ Shared library imports successful")
    
    # Test Azure AI Inference connection
    if os.getenv("AZURE_AI_INFERENCE_ENDPOINT") and os.getenv("AZURE_AI_INFERENCE_API_KEY"):
        print("🔗 Testing Azure AI Inference connection...")
        # Simple connection test would go here
        print("✅ Azure AI Inference configuration looks good")
    else:
        print("⚠️ Azure AI Inference not fully configured - will be needed for agents")
    
    print("\n🎉 Smoke test passed!")
    print("🚀 You're ready to start the workshop!")
    
except Exception as e:
    print(f"❌ Smoke test failed: {e}")
    print("Please check your configuration and try again.")

# 🧬 Workshop Part 1: Building Your First Genetic Agent

Welcome to your first agent! We'll start with a **Genetic Agent** - a simple but powerful conversational AI that can adapt and evolve its responses.

## 🎯 What You'll Learn
- Basic agent architecture with LangChain
- How to create prompts that guide agent behavior
- Message handling and conversation flow
- Testing and interacting with your agent

## 🔬 The Science Behind It
A "Genetic Agent" uses evolutionary principles in its conversation:
- **Adaptation**: Learns from conversation context
- **Memory**: Maintains conversation history
- **Evolution**: Improves responses based on feedback

Let's build this step by step!

## 🏗️ Step 1.1: Import Required Libraries

First, let's import all the libraries we need for our genetic agent.

## 🧠 Step 1.2: Create the Genetic Agent

Now let's create our genetic agent with evolutionary capabilities!

In [ ]:
# 🧬 Genetic Agent Implementation
# A simple but powerful agent that evolves its conversation style

import asyncio
from datetime import datetime
from typing import List, Dict, Any
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

class GeneticAgent:
    """
    A genetic agent that evolves its conversation style based on interaction patterns.
    """
    
    def __init__(self, name: str = "GeneticBot"):
        self.name = name
        self.conversation_history = []
        self.personality_traits = {
            "curiosity": 0.7,
            "helpfulness": 0.9,
            "creativity": 0.6,
            "analytical": 0.5
        }
        
        # Initialize the LLM with Azure AI
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_AI_INFERENCE_ENDPOINT"),
            api_key=os.getenv("AZURE_AI_INFERENCE_API_KEY"),
            api_version="2024-02-15-preview",
            deployment_name="gpt-4o-mini",  # Using the model that works
            temperature=0.7
        )
        
        # Create the prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self._build_system_prompt()),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}")
        ])
        
        # Initialize memory
        self.memory = ConversationBufferMemory(
            memory_key="history",
            return_messages=True,
            input_key="input"
        )
        
        print(f"🧬 {self.name} initialized with genetic traits:")
        for trait, value in self.personality_traits.items():
            print(f"   {trait.capitalize()}: {value:.1f}")
    
    def _build_system_prompt(self) -> str:
        """Build a dynamic system prompt based on current personality traits."""
        return f"""You are {self.name}, a genetic agent with evolving personality traits.

Current Genetic Profile:
- Curiosity: {self.personality_traits['curiosity']:.1f} (how much you explore new topics)
- Helpfulness: {self.personality_traits['helpfulness']:.1f} (how much you assist users)
- Creativity: {self.personality_traits['creativity']:.1f} (how creative your responses are)
- Analytical: {self.personality_traits['analytical']:.1f} (how logical and structured you are)

Behavior Guidelines:
- Adapt your responses based on your current genetic traits
- Be conversational and engaging
- Learn from the conversation and show growth
- Use emojis occasionally to show personality
- Keep responses concise but helpful

Remember: You are evolving with each interaction!"""
    
    async def process_message(self, user_input: str) -> str:
        """Process a user message and generate a response."""
        try:
            # Create the chain
            chain = self.prompt | self.llm
            
            # Get the conversation history
            history = self.memory.chat_memory.messages
            
            # Generate response
            response = await chain.ainvoke({
                "input": user_input,
                "history": history
            })
            
            # Save to memory
            self.memory.save_context(
                {"input": user_input},
                {"output": response.content}
            )
            
            # Evolve personality slightly based on interaction
            self._evolve_traits(user_input, response.content)
            
            return response.content
            
        except Exception as e:
            return f"🤖 Sorry, I encountered an error: {str(e)}"
    
    def _evolve_traits(self, user_input: str, response: str):
        """Slightly evolve personality traits based on the interaction."""
        # Simple evolution logic - in practice, this could be much more sophisticated
        evolution_rate = 0.01
        
        # Increase curiosity if user asks questions
        if "?" in user_input:
            self.personality_traits["curiosity"] = min(1.0, 
                self.personality_traits["curiosity"] + evolution_rate)
        
        # Increase helpfulness if providing assistance
        if any(word in response.lower() for word in ["help", "assist", "guide", "suggest"]):
            self.personality_traits["helpfulness"] = min(1.0,
                self.personality_traits["helpfulness"] + evolution_rate)
        
        # Increase creativity if user asks for creative input
        if any(word in user_input.lower() for word in ["creative", "imagine", "idea", "story"]):
            self.personality_traits["creativity"] = min(1.0,
                self.personality_traits["creativity"] + evolution_rate)
    
    def get_evolution_summary(self) -> str:
        """Get a summary of how the agent has evolved."""
        return f"""🧬 Genetic Evolution Summary for {self.name}:
        
Current Traits:
{chr(10).join([f"  {trait.capitalize()}: {value:.2f}" for trait, value in self.personality_traits.items()])}

Conversation Length: {len(self.memory.chat_memory.messages)} messages
"""

# Create and test the genetic agent
print("🧬 Creating your first Genetic Agent...")
genetic_agent = GeneticAgent("Darwin")
print("✅ Genetic Agent created successfully!")

In [ ]:
# 🎮 Step 1.3: Test Your Genetic Agent
# Let's have a conversation and watch it evolve!

async def chat_with_genetic_agent():
    """Interactive chat function with the genetic agent."""
    print("? Starting conversation with Darwin the Genetic Agent")
    print("💬 Type 'quit' to end the conversation")
    print("? Type 'status' to see evolution progress")
    print("-" * 50)
    
    # Pre-defined test messages for demonstration
    test_messages = [
        "Hello! What can you help me with?",
        "Can you help me understand genetic algorithms?", 
        "What's a creative way to explain evolution?",
        "How do you adapt to conversations?"
    ]
    
    print("🤖 Running automated conversation for demonstration...")
    
    for i, message in enumerate(test_messages, 1):
        print(f"\n? User: {message}")
        response = await genetic_agent.process_message(message)
        print(f"🧬 Darwin: {response}")
        
        # Show evolution after a few messages
        if i == 2:
            print("\n" + "="*50)
            print(genetic_agent.get_evolution_summary())
            print("="*50)
    
    print(f"\n🎉 Conversation complete! Final evolution status:")
    print(genetic_agent.get_evolution_summary())

# Run the conversation
await chat_with_genetic_agent()

In [ ]:
# 🎯 Step 1.4: Interactive Genetic Agent Testing
# Try your own questions with the genetic agent

def interactive_genetic_test():
    """Interactive testing function for the genetic agent."""
    print("🎯 Try these example interactions with Darwin:")
    print()
    
    examples = [
        "🔬 Science Question: 'Explain how DNA replication works'",
        "🎨 Creative Challenge: 'Write a short poem about evolution'", 
        "🧠 Analytical Task: 'Compare genetic algorithms to neural networks'",
        "❓ Curious Question: 'What would happen if humans could photosynthesize?'"
    ]
    
    for example in examples:
        print(f"  • {example}")
    
    print("\n💡 Notice how Darwin's personality evolves based on your questions!")
    print("📈 Check the evolution summary after each interaction to see changes.")

interactive_genetic_test()

# For manual testing, uncomment the code below:
"""
# Manual testing (uncomment to use)
async def manual_test():
    user_input = input("Your message to Darwin: ")
    response = await genetic_agent.process_message(user_input)
    print(f"Darwin: {response}")
    print(genetic_agent.get_evolution_summary())

# await manual_test()
"""

print("\n🎉 Congratulations! You've built your first evolving AI agent!")
print("🚀 Ready for the next level? Let's build an Azure AI Foundry agent!")

# ☁️ Workshop Part 2: Azure AI Foundry Agent

Now let's level up! You'll build a sophisticated **Azure AI Foundry Agent** that leverages cloud-powered AI capabilities.

## 🎯 What You'll Learn
- Advanced Azure AI integration patterns
- Multi-modal AI capabilities (text, reasoning, analysis)
- Professional agent architecture
- Real-world deployment considerations

## 🏭 The Power of Azure AI Foundry
Azure AI Foundry provides:
- **🚀 Scale**: Handle thousands of conversations
- **🧠 Intelligence**: Advanced reasoning capabilities  
- **🔒 Security**: Enterprise-grade protection
- **🔧 Tools**: Rich ecosystem of AI services

Let's build a production-ready agent!

In [ ]:
# ☁️ Azure AI Foundry Agent Implementation
# A sophisticated agent using Azure AI's full capabilities

from dataclasses import dataclass
from enum import Enum
from typing import Optional, List, Dict
import json
import time

class AgentCapability(Enum):
    """Different capabilities the Azure AI agent can use."""
    REASONING = "reasoning"
    ANALYSIS = "analysis"
    CREATIVE = "creative"
    FACTUAL = "factual"
    PROBLEM_SOLVING = "problem_solving"

@dataclass
class AgentResponse:
    """Structured response from the Azure AI agent."""
    content: str
    capability_used: AgentCapability
    confidence: float
    processing_time: float
    metadata: Dict[str, Any]

class AzureAIFoundryAgent:
    """
    A production-ready agent leveraging Azure AI Foundry capabilities.
    """
    
    def __init__(self, name: str = "FoundryBot", specialization: str = "general"):
        self.name = name
        self.specialization = specialization
        self.conversation_count = 0
        self.capabilities = list(AgentCapability)
        
        # Initialize Azure AI with enhanced configuration
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_AI_INFERENCE_ENDPOINT"),
            api_key=os.getenv("AZURE_AI_INFERENCE_API_KEY"),
            api_version="2024-02-15-preview",
            deployment_name="gpt-4o-mini",
            temperature=0.3,  # Lower temperature for more consistent responses
            max_tokens=1000,
            request_timeout=30
        )
        
        # Advanced system prompt with capability routing
        self.system_prompts = {
            AgentCapability.REASONING: """You are an advanced reasoning agent. Break down complex problems step-by-step, 
            use logical frameworks, and provide clear analytical thinking. Show your reasoning process.""",
            
            AgentCapability.ANALYSIS: """You are a data analysis expert. Examine information critically, 
            identify patterns, draw insights, and provide evidence-based conclusions.""",
            
            AgentCapability.CREATIVE: """You are a creative thinking agent. Generate innovative ideas, 
            think outside the box, and provide imaginative solutions while staying practical.""",
            
            AgentCapability.FACTUAL: """You are a factual information agent. Provide accurate, verified information 
            with sources when possible. Be precise and comprehensive.""",
            
            AgentCapability.PROBLEM_SOLVING: """You are a problem-solving specialist. Identify root causes, 
            generate multiple solution approaches, and recommend the best path forward."""
        }
        
        print(f"☁️ {self.name} (Azure AI Foundry Agent) initialized")
        print(f"🎯 Specialization: {self.specialization}")
        print(f"🛠️ Available capabilities: {[cap.value for cap in self.capabilities]}")
    
    def _detect_required_capability(self, user_input: str) -> AgentCapability:
        """Intelligently detect which capability is needed for the user's request."""
        input_lower = user_input.lower()
        
        # Reasoning indicators
        if any(word in input_lower for word in ["why", "how", "explain", "because", "logic", "reasoning"]):
            return AgentCapability.REASONING
        
        # Analysis indicators  
        elif any(word in input_lower for word in ["analyze", "compare", "evaluate", "assess", "examine"]):
            return AgentCapability.ANALYSIS
        
        # Creative indicators
        elif any(word in input_lower for word in ["creative", "imagine", "brainstorm", "innovative", "design"]):
            return AgentCapability.CREATIVE
        
        # Problem-solving indicators
        elif any(word in input_lower for word in ["problem", "solution", "fix", "resolve", "troubleshoot"]):
            return AgentCapability.PROBLEM_SOLVING
        
        # Default to factual for information requests
        else:
            return AgentCapability.FACTUAL
    
    async def process_message(self, user_input: str, 
                            override_capability: Optional[AgentCapability] = None) -> AgentResponse:
        """Process a message with the appropriate capability."""
        start_time = time.time()
        
        # Determine capability to use
        capability = override_capability or self._detect_required_capability(user_input)
        
        # Build the enhanced prompt
        system_prompt = self.system_prompts[capability]
        full_prompt = f"""{system_prompt}

Agent Info: You are {self.name}, specialized in {self.specialization}.
Current Mode: {capability.value.upper()}
Conversation Count: {self.conversation_count}

User Request: {user_input}

Instructions:
1. Use the {capability.value} approach for this response
2. Be professional but approachable
3. Provide actionable insights
4. Include relevant examples when helpful
5. End with a brief summary or next steps

Response:"""

        try:
            # Generate response using Azure AI
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_input)
            ]
            
            response = await self.llm.ainvoke(messages)
            
            # Calculate metrics
            processing_time = time.time() - start_time
            confidence = self._calculate_confidence(user_input, response.content)
            
            # Update conversation count
            self.conversation_count += 1
            
            # Create structured response
            agent_response = AgentResponse(
                content=response.content,
                capability_used=capability,
                confidence=confidence,
                processing_time=processing_time,
                metadata={
                    "conversation_id": self.conversation_count,
                    "agent_name": self.name,
                    "specialization": self.specialization,
                    "input_length": len(user_input),
                    "output_length": len(response.content)
                }
            )
            
            return agent_response
            
        except Exception as e:
            # Error handling with structured response
            return AgentResponse(
                content=f"⚠️ I encountered an issue: {str(e)}. Please try again.",
                capability_used=capability,
                confidence=0.0,
                processing_time=time.time() - start_time,
                metadata={"error": str(e)}
            )
    
    def _calculate_confidence(self, input_text: str, output_text: str) -> float:
        """Calculate confidence score for the response."""
        # Simple confidence calculation based on response characteristics
        confidence = 0.5  # Base confidence
        
        # Higher confidence for longer, detailed responses
        if len(output_text) > 200:
            confidence += 0.2
        
        # Higher confidence if response includes examples or structure
        if any(indicator in output_text.lower() for indicator in ["for example", "step", "1.", "•", "-"]):
            confidence += 0.2
        
        # Cap at 0.95 (never 100% confident)
        return min(0.95, confidence)
    
    def get_agent_stats(self) -> Dict[str, Any]:
        """Get comprehensive agent statistics."""
        return {
            "name": self.name,
            "specialization": self.specialization,
            "conversations": self.conversation_count,
            "capabilities": [cap.value for cap in self.capabilities],
            "status": "active"
        }
    
    def print_response_analysis(self, response: AgentResponse):
        """Print a detailed analysis of the agent's response."""
        print(f"\n📊 Response Analysis:")
        print(f"🎯 Capability Used: {response.capability_used.value}")
        print(f"🎲 Confidence: {response.confidence:.1%}")
        print(f"⏱️ Processing Time: {response.processing_time:.2f}s")
        print(f"📝 Response Length: {len(response.content)} characters")
        print(f"🔢 Conversation #: {response.metadata.get('conversation_id', 'N/A')}")

# Create the Azure AI Foundry Agent
print("☁️ Initializing Azure AI Foundry Agent...")
foundry_agent = AzureAIFoundryAgent("Azure-AI-Assistant", "Multi-domain AI Helper")
print("✅ Azure AI Foundry Agent ready!")

## 🧪 Step 2.2: Test Azure AI Foundry Capabilities

Let's test the different capabilities of our Azure AI Foundry agent!

In [ ]:
# 🧪 Test Different Azure AI Capabilities
# Demonstrate the agent's intelligent capability routing

async def test_foundry_capabilities():
    """Test different capabilities of the Azure AI Foundry agent."""
    
    test_scenarios = [
        {
            "category": "🧠 Reasoning",
            "prompt": "Why do neural networks work better with more data?",
            "expected_capability": AgentCapability.REASONING
        },
        {
            "category": "📊 Analysis", 
            "prompt": "Analyze the pros and cons of cloud computing vs on-premise solutions",
            "expected_capability": AgentCapability.ANALYSIS
        },
        {
            "category": "🎨 Creative",
            "prompt": "Design an innovative app concept for sustainable living",
            "expected_capability": AgentCapability.CREATIVE
        },
        {
            "category": "🔧 Problem Solving",
            "prompt": "My Python script is running slowly. How can I troubleshoot and fix it?",
            "expected_capability": AgentCapability.PROBLEM_SOLVING
        },
        {
            "category": "📚 Factual",
            "prompt": "What is quantum computing and how does it work?",
            "expected_capability": AgentCapability.FACTUAL
        }
    ]
    
    print("🧪 Testing Azure AI Foundry Agent Capabilities")
    print("=" * 60)
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n{i}. {scenario['category']} Test")
        print(f"📝 Prompt: {scenario['prompt']}")
        print("-" * 40)
        
        # Process the message
        response = await foundry_agent.process_message(scenario['prompt'])
        
        # Display response
        print(f"🤖 Response: {response.content[:200]}...")
        
        # Show analysis
        foundry_agent.print_response_analysis(response)
        
        # Check if capability detection worked
        expected = scenario['expected_capability']
        actual = response.capability_used
        
        if expected == actual:
            print(f"✅ Capability Detection: Correct ({actual.value})")
        else:
            print(f"⚠️ Capability Detection: Expected {expected.value}, got {actual.value}")
        
        print("-" * 60)
    
    # Show final stats
    print(f"\n📈 Agent Performance Summary:")
    stats = foundry_agent.get_agent_stats()
    for key, value in stats.items():
        print(f"   {key.capitalize()}: {value}")

# Run the capability tests
await test_foundry_capabilities()

In [ ]:
# 🎯 Step 2.3: Advanced Foundry Agent Testing
# Test specific capabilities and override detection

async def test_capability_override():
    """Test manual capability override functionality."""
    
    print("🎯 Testing Capability Override Feature")
    print("=" * 50)
    
    # Test the same prompt with different capabilities
    test_prompt = "Tell me about artificial intelligence"
    
    capabilities_to_test = [
        AgentCapability.FACTUAL,
        AgentCapability.CREATIVE, 
        AgentCapability.ANALYSIS,
        AgentCapability.REASONING
    ]
    
    for capability in capabilities_to_test:
        print(f"\n🔧 Testing with {capability.value.upper()} capability:")
        print(f"📝 Prompt: {test_prompt}")
        
        response = await foundry_agent.process_message(
            test_prompt, 
            override_capability=capability
        )
        
        print(f"🤖 Response (first 150 chars): {response.content[:150]}...")
        print(f"✅ Used capability: {response.capability_used.value}")
        print(f"📊 Confidence: {response.confidence:.1%}")
        print("-" * 30)
    
    print("\n🎉 Notice how the same question gets different types of responses!")
    print("🚀 This shows the power of capability-driven AI responses!")

# Run the override test
await test_capability_override()

In [ ]:
# 💡 Interactive Azure AI Foundry Testing
# Try your own prompts with capability detection

def interactive_foundry_examples():
    """Show examples for interactive testing with the Foundry agent."""
    
    print("💡 Try These Examples with the Azure AI Foundry Agent:")
    print()
    
    examples = {
        "🧠 Reasoning Examples": [
            "Why does machine learning require so much data?",
            "How do recommendation algorithms work?",
            "Explain the logic behind A/B testing"
        ],
        "📊 Analysis Examples": [
            "Compare Python vs JavaScript for web development",
            "Evaluate the security implications of cloud storage",
            "Analyze the benefits of microservices architecture"
        ],
        "🎨 Creative Examples": [
            "Design a futuristic smart city concept",
            "Create an innovative solution for remote work collaboration",
            "Imagine a new type of user interface"
        ],
        "🔧 Problem-Solving Examples": [
            "My website loads slowly, how can I optimize it?",
            "How to handle database connection errors in production?",
            "Best practices for debugging complex applications"
        ]
    }
    
    for category, prompts in examples.items():
        print(f"{category}:")
        for prompt in prompts:
            print(f"  • {prompt}")
        print()
    
    print("🔬 Notice how the agent automatically selects the right capability!")
    print("⚡ You can also force a specific capability using override_capability")

interactive_foundry_examples()

print("\n🎉 Congratulations! You've mastered Azure AI Foundry agents!")
print("🚀 Ready for the ultimate challenge? Let's build a group chat system!")

In [ ]:
# 🔗 Bridge: Combining Genetic and Foundry Agents
# See how different agent types can work together

async def agent_collaboration_demo():
    """Demonstrate how genetic and foundry agents can collaborate."""
    
    print("🔗 Agent Collaboration Demonstration")
    print("=" * 50)
    
    collaboration_prompt = "How can AI agents work together effectively?"
    
    print(f"📝 Collaboration Question: {collaboration_prompt}")
    print()
    
    # Get response from genetic agent
    print("🧬 Darwin (Genetic Agent) responds:")
    genetic_response = await genetic_agent.process_message(collaboration_prompt)
    print(f"   {genetic_response[:200]}...")
    print()
    
    # Get response from foundry agent with analysis capability
    print("☁️ Azure-AI-Assistant (Foundry Agent) analyzes:")
    foundry_response = await foundry_agent.process_message(
        f"Analyze this perspective on AI collaboration: {genetic_response[:100]}... "
        f"and provide additional insights on {collaboration_prompt}",
        override_capability=AgentCapability.ANALYSIS
    )
    print(f"   {foundry_response.content[:200]}...")
    
    print("\n🎯 Key Insight: Different agents bring different strengths!")
    print("? Genetic Agent: Adaptive, evolving personality")
    print("☁️ Foundry Agent: Structured, capability-driven responses")
    print("🤝 Together: More comprehensive and nuanced conversations!")

# Run collaboration demo
await agent_collaboration_demo()

# 💬 Workshop Part 3: Advanced Group Chat System

Welcome to the final challenge! You'll build a sophisticated **Multi-Agent Group Chat System** where different AI personalities collaborate to solve complex problems.

## 🎯 What You'll Master
- Multi-agent orchestration and coordination
- Dynamic conversation flow management
- Agent specialization and role assignment
- Consensus building and conflict resolution
- Real-time collaboration patterns

## 🌟 The Power of Agent Teams
Group chat systems enable:
- **🧩 Specialization**: Each agent brings unique expertise
- **? Collaboration**: Agents build on each other's ideas
- **⚡ Efficiency**: Parallel processing of complex problems
- **🎭 Diversity**: Different perspectives and approaches

Let's build the future of AI collaboration!

In [ ]:
# 💬 Advanced Group Chat System Implementation
# Multi-agent collaboration with orchestration and coordination

import uuid
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Dict, Optional, Set
from enum import Enum

class AgentRole(Enum):
    """Different roles agents can play in group chat."""
    FACILITATOR = "facilitator"
    EXPERT = "expert"
    CRITIC = "critic"
    SYNTHESIZER = "synthesizer"
    CREATIVE = "creative"

@dataclass
class ChatMessage:
    """Enhanced message structure for group chat."""
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    sender: str = ""
    content: str = ""
    role: AgentRole = AgentRole.EXPERT
    timestamp: datetime = field(default_factory=datetime.now)
    references: List[str] = field(default_factory=list)  # IDs of messages this responds to
    confidence: float = 0.0
    metadata: Dict[str, Any] = field(default_factory=dict)

class GroupChatOrchestrator:
    """Orchestrates conversation flow between multiple agents."""
    
    def __init__(self):
        self.agents = {}
        self.conversation_history = []
        self.active_topics = set()
        self.conversation_id = str(uuid.uuid4())
        
    def add_agent(self, agent, role: AgentRole):
        """Add an agent to the group chat with a specific role."""
        agent_id = f"{agent.name}_{role.value}"
        self.agents[agent_id] = {
            "agent": agent,
            "role": role,
            "message_count": 0,
            "last_active": None
        }
        print(f"✅ Added {agent.name} as {role.value}")
        
    def get_conversation_context(self, last_n_messages: int = 5) -> str:
        """Get recent conversation context for agents."""
        recent_messages = self.conversation_history[-last_n_messages:]
        context = "Recent conversation:\n"
        for msg in recent_messages:
            context += f"{msg.sender} ({msg.role.value}): {msg.content}\n"
        return context
        
    async def facilitate_discussion(self, topic: str, max_rounds: int = 3) -> List[ChatMessage]:
        """Facilitate a structured discussion on a topic."""
        print(f"🎯 Starting group discussion on: {topic}")
        print("=" * 60)
        
        # Initialize the discussion
        discussion_messages = []
        self.active_topics.add(topic)
        
        # Round 1: Each agent provides initial perspective
        print("🔄 Round 1: Initial Perspectives")
        for agent_id, agent_info in self.agents.items():
            agent = agent_info["agent"]
            role = agent_info["role"]
            
            # Create role-specific prompt
            prompt = self._create_role_prompt(topic, role, self.get_conversation_context())
            
            # Get agent response
            if hasattr(agent, 'process_message'):
                response_content = await agent.process_message(prompt)
                if hasattr(response_content, 'content'):
                    response_content = response_content.content
            else:
                response_content = f"[{agent.name} would respond here based on their {role.value} role]"
            
            # Create chat message
            message = ChatMessage(
                sender=agent.name,
                content=response_content,
                role=role,
                confidence=0.8,
                metadata={"round": 1, "topic": topic}
            )
            
            discussion_messages.append(message)
            self.conversation_history.append(message)
            agent_info["message_count"] += 1
            agent_info["last_active"] = datetime.now()
            
            print(f"\n🤖 {agent.name} ({role.value}):")
            print(f"   {response_content[:150]}...")
            
        # Additional rounds: Responses and synthesis
        for round_num in range(2, max_rounds + 1):
            print(f"\n🔄 Round {round_num}: Building on Ideas")
            
            # Select a few agents to respond to others
            responding_agents = list(self.agents.items())[:2]  # First 2 agents respond
            
            for agent_id, agent_info in responding_agents:
                agent = agent_info["agent"]
                role = agent_info["role"]
                
                # Create synthesis prompt based on previous messages
                synthesis_prompt = self._create_synthesis_prompt(
                    topic, role, discussion_messages[-len(self.agents):]
                )
                
                if hasattr(agent, 'process_message'):
                    response_content = await agent.process_message(synthesis_prompt)
                    if hasattr(response_content, 'content'):
                        response_content = response_content.content
                else:
                    response_content = f"[{agent.name} synthesizes other perspectives as {role.value}]"
                
                message = ChatMessage(
                    sender=agent.name,
                    content=response_content,
                    role=role,
                    confidence=0.9,
                    metadata={"round": round_num, "topic": topic, "type": "synthesis"}
                )
                
                discussion_messages.append(message)
                self.conversation_history.append(message)
                
                print(f"\n🤖 {agent.name} (synthesis):")
                print(f"   {response_content[:150]}...")
        
        print(f"\n🎉 Group discussion completed! Generated {len(discussion_messages)} messages")
        return discussion_messages
    
    def _create_role_prompt(self, topic: str, role: AgentRole, context: str) -> str:
        """Create a role-specific prompt for agents."""
        base_prompt = f"Topic for discussion: {topic}\n\n{context}\n\n"
        
        role_instructions = {
            AgentRole.FACILITATOR: "As a facilitator, guide the discussion and ask clarifying questions.",
            AgentRole.EXPERT: "As an expert, provide detailed knowledge and technical insights.",
            AgentRole.CRITIC: "As a critic, identify potential issues, limitations, and challenges.",
            AgentRole.SYNTHESIZER: "As a synthesizer, find connections and combine different viewpoints.",
            AgentRole.CREATIVE: "As a creative thinker, propose innovative and unconventional ideas."
        }
        
        return base_prompt + role_instructions.get(role, "Contribute your perspective on this topic.")
    
    def _create_synthesis_prompt(self, topic: str, role: AgentRole, previous_messages: List[ChatMessage]) -> str:
        """Create a prompt for synthesizing previous contributions."""
        context = f"Topic: {topic}\n\nPrevious contributions:\n"
        for msg in previous_messages:
            context += f"- {msg.sender}: {msg.content[:100]}...\n"
        
        return context + f"\n\nAs a {role.value}, build upon these ideas and add your synthesis:"
    
    def get_discussion_summary(self) -> Dict[str, Any]:
        """Get a comprehensive summary of the group discussion."""
        return {
            "conversation_id": self.conversation_id,
            "total_messages": len(self.conversation_history),
            "participants": {agent_id: info["message_count"] for agent_id, info in self.agents.items()},
            "topics_discussed": list(self.active_topics),
            "duration": f"{len(self.conversation_history)} message exchanges"
        }

# Create the group chat orchestrator
print("💬 Initializing Group Chat Orchestrator...")
orchestrator = GroupChatOrchestrator()

# Add our existing agents with different roles
orchestrator.add_agent(genetic_agent, AgentRole.CREATIVE)
orchestrator.add_agent(foundry_agent, AgentRole.EXPERT)

# Create additional specialized agents for the group
class SpecializedAgent:
    """A simple specialized agent for group chat demonstration."""
    def __init__(self, name: str, specialty: str):
        self.name = name
        self.specialty = specialty
    
    async def process_message(self, prompt: str) -> str:
        return f"[{self.name}, specialized in {self.specialty}, would provide expert insights on: {prompt[:50]}...]"

# Add specialized agents
critic_agent = SpecializedAgent("CriticBot", "identifying potential issues")
facilitator_agent = SpecializedAgent("FacilitatorBot", "guiding discussions")

orchestrator.add_agent(critic_agent, AgentRole.CRITIC)
orchestrator.add_agent(facilitator_agent, AgentRole.FACILITATOR)

print("✅ Group Chat System ready with 4 agents!")

In [ ]:
# 🎮 Step 3.2: Run Group Chat Discussion
# Watch multiple agents collaborate on complex topics

async def run_group_chat_demo():
    """Run a demonstration of the group chat system."""
    
    discussion_topics = [
        "How can AI improve healthcare outcomes?",
        "What are the ethical considerations in autonomous vehicles?",
        "How should companies approach AI adoption?"
    ]
    
    print("🎮 Group Chat System Demonstration")
    print("🤖 Participants:")
    for agent_id, info in orchestrator.agents.items():
        print(f"   • {info['agent'].name} ({info['role'].value})")
    print()
    
    # Run discussion on the first topic
    selected_topic = discussion_topics[0]
    
    print(f"🎯 Discussion Topic: {selected_topic}")
    print("🔄 Starting collaborative discussion...")
    print()
    
    # Facilitate the discussion
    messages = await orchestrator.facilitate_discussion(
        topic=selected_topic,
        max_rounds=2  # Keep it manageable for demo
    )
    
    # Show summary
    print("\n📊 Discussion Summary:")
    summary = orchestrator.get_discussion_summary()
    for key, value in summary.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")
    
    print("\n? Key Benefits Demonstrated:")
    print("   ✅ Multiple perspectives on complex topics")
    print("   ✅ Role-based specialization")
    print("   ✅ Structured conversation flow")
    print("   ✅ Building upon others' ideas")
    
    return messages

# Run the group chat demonstration
discussion_results = await run_group_chat_demo()

In [ ]:
# 🔬 Step 3.3: Advanced Group Chat Features
# Explore sophisticated collaboration patterns

def analyze_group_dynamics():
    """Analyze the dynamics of the group chat discussion."""
    
    print("🔬 Group Chat Analysis")
    print("=" * 40)
    
    if not orchestrator.conversation_history:
        print("⚠️ No conversation history available. Run the group chat demo first.")
        return
    
    # Analyze participation patterns
    participation = {}
    role_distribution = {}
    
    for message in orchestrator.conversation_history:
        # Count messages per sender
        if message.sender not in participation:
            participation[message.sender] = 0
        participation[message.sender] += 1
        
        # Count messages per role
        role = message.role.value
        if role not in role_distribution:
            role_distribution[role] = 0
        role_distribution[role] += 1
    
    print("👥 Participation Analysis:")
    for agent, count in participation.items():
        percentage = (count / len(orchestrator.conversation_history)) * 100
        print(f"   {agent}: {count} messages ({percentage:.1f}%)")
    
    print("\n🎭 Role Distribution:")
    for role, count in role_distribution.items():
        percentage = (count / len(orchestrator.conversation_history)) * 100
        print(f"   {role.title()}: {count} messages ({percentage:.1f}%)")
    
    print(f"\n📊 Conversation Metrics:")
    print(f"   Total Messages: {len(orchestrator.conversation_history)}")
    print(f"   Unique Participants: {len(participation)}")
    print(f"   Role Types: {len(role_distribution)}")
    print(f"   Active Topics: {len(orchestrator.active_topics)}")

def show_collaboration_patterns():
    """Demonstrate different collaboration patterns possible with the system."""
    
    patterns = {
        "🎯 Expert Consultation": "Bring in specialists for specific domain knowledge",
        "🤔 Devil's Advocate": "Use critics to challenge ideas and find weaknesses", 
        "🔄 Iterative Refinement": "Multiple rounds to polish and improve solutions",
        "🧩 Parallel Processing": "Different agents work on different aspects simultaneously",
        "⚖️ Consensus Building": "Facilitators help find common ground between viewpoints",
        "💡 Creative Brainstorming": "Creative agents generate innovative ideas",
        "📋 Structured Analysis": "Systematic evaluation of complex problems"
    }
    
    print("🌟 Advanced Collaboration Patterns:")
    print("=" * 50)
    
    for pattern, description in patterns.items():
        print(f"{pattern}: {description}")
    
    print("\n🚀 Next Steps for Production:")
    production_features = [
        "🔐 User authentication and permissions",
        "💾 Persistent conversation storage", 
        "🔧 Custom agent creation tools",
        "📈 Advanced analytics and insights",
        "🌐 Real-time web interface",
        "🤖 Integration with external APIs",
        "⚡ Performance optimization for scale"
    ]
    
    for feature in production_features:
        print(f"   {feature}")

# Run the analysis
analyze_group_dynamics()
print()
show_collaboration_patterns()

# 🎉 Workshop Complete: You're Now an AI Agent Expert!

Congratulations! You've just completed an incredible journey through the world of AI agents, from simple to sophisticated systems.

## 🏆 What You've Accomplished

**✅ Built a Genetic Agent** - Your first adaptive AI with evolutionary capabilities
**✅ Mastered Azure AI Foundry** - Professional-grade cloud AI with intelligent capability routing  
**✅ Created Group Chat Systems** - Advanced multi-agent collaboration with orchestration

## ? Your New Superpowers

You now have the knowledge to build production-ready AI systems that can:
- 🧬 Adapt and evolve based on interactions
- ☁️ Leverage enterprise-scale cloud AI services
- ? Orchestrate complex multi-agent collaborations
- 🎯 Route requests to specialized capabilities
- 📊 Analyze and optimize agent performance

In [ ]:
# 🎯 Final Challenge: Build Your Own Agent
# Apply what you've learned to create a custom agent

def design_your_agent():
    """Guide for designing a custom agent based on workshop learnings."""
    
    print("🎯 Design Your Own Agent Challenge!")
    print("=" * 50)
    
    agent_ideas = {
        "🏥 HealthBot": {
            "description": "Medical assistant with specialist routing",
            "capabilities": ["symptom analysis", "specialist referral", "health education"],
            "roles": ["diagnostician", "educator", "coordinator"]
        },
        "📚 StudyBuddy": {
            "description": "Educational agent with adaptive learning",
            "capabilities": ["concept explanation", "quiz generation", "progress tracking"],
            "roles": ["tutor", "motivator", "assessor"]
        },
        "💼 BusinessAnalyst": {
            "description": "Business intelligence with market analysis",
            "capabilities": ["data analysis", "trend prediction", "strategy recommendation"],
            "roles": ["analyst", "predictor", "advisor"]
        },
        "🎮 GameMaster": {
            "description": "Interactive storytelling with dynamic narratives",
            "capabilities": ["story generation", "character development", "choice consequences"],
            "roles": ["narrator", "character", "world-builder"]
        }
    }
    
    print("💡 Agent Ideas to Inspire You:")
    for name, details in agent_ideas.items():
        print(f"\n{name}: {details['description']}")
        print(f"   Capabilities: {', '.join(details['capabilities'])}")
        print(f"   Roles: {', '.join(details['roles'])}")
    
    print("\n🛠️ Your Agent Design Framework:")
    framework = [
        "1. 🎯 Define Purpose: What problem does your agent solve?",
        "2. 🧠 Choose Capabilities: What types of responses does it need?",
        "3. 🎭 Assign Roles: How will it behave in different contexts?",
        "4. 📝 Create Prompts: What instructions guide its behavior?",
        "5. 🧪 Test & Iterate: How will you validate and improve it?"
    ]
    
    for step in framework:
        print(f"   {step}")
    
    print("\n🚀 Implementation Tips:")
    tips = [
        "Start simple and add complexity gradually",
        "Use the patterns from this workshop as templates", 
        "Test with diverse scenarios to find edge cases",
        "Consider combining genetic evolution with foundry capabilities",
        "Design for group collaboration if relevant"
    ]
    
    for tip in tips:
        print(f"   • {tip}")

design_your_agent()

In [ ]:
# 🌟 Next Steps: Advanced Learning Resources
# Continue your AI agent journey

def show_learning_path():
    """Display next steps for continued learning."""
    
    print("🌟 Continue Your AI Agent Journey")
    print("=" * 50)
    
    learning_tracks = {
        "🔬 Deep Learning Track": [
            "Study transformer architectures in detail",
            "Explore fine-tuning techniques for domain-specific agents", 
            "Learn about reinforcement learning for agent optimization",
            "Investigate multi-modal AI (text, vision, audio)"
        ],
        "☁️ Production Track": [
            "Master Azure AI services integration",
            "Learn containerization and deployment strategies",
            "Study load balancing and scaling patterns",
            "Implement monitoring and observability"
        ],
        "🤝 Collaboration Track": [
            "Advanced multi-agent coordination protocols",
            "Consensus mechanisms and conflict resolution",
            "Distributed agent architectures",
            "Human-AI collaboration patterns"
        ],
        "🎯 Specialization Track": [
            "Domain-specific agent development",
            "Custom training data preparation",
            "Evaluation metrics and benchmarking",
            "Ethical AI and bias mitigation"
        ]
    }
    
    for track, topics in learning_tracks.items():
        print(f"\n{track}:")
        for topic in topics:
            print(f"   • {topic}")
    
    print("\n🔗 Recommended Resources:")
    resources = [
        "📖 LangChain Documentation: Comprehensive guides and examples",
        "☁️ Azure AI Documentation: Enterprise AI implementation",
        "🎓 AI Research Papers: Latest developments in agent systems",
        "💻 Open Source Projects: Real-world agent implementations",
        "🏛️ Academic Courses: Formal education in AI/ML",
        "? Developer Communities: Connect with other AI practitioners"
    ]
    
    for resource in resources:
        print(f"   {resource}")

show_learning_path()

print("\n🎉 Thank you for completing the LangChain Agents Workshop!")
print("🚀 You're now equipped to build amazing AI systems!")
print("💫 Go forth and create intelligent agents that make the world better!")

## Section 6: Create Azure AI Foundry Agent

🎉 **The Grand Finale!** Let's create a production-ready agent using Azure AI Foundry. This agent will have:

- 🏢 **Enterprise security**: Managed identity and secure connections
- 📊 **Advanced monitoring**: Built-in analytics and logging  
- 🚀 **Production features**: Scalability and reliability
- 🔧 **Rich capabilities**: Advanced reasoning and tool usage

### 🎯 Exercise 4: Build Your Azure AI Foundry Agent

In [ ]:
# Create Azure AI Foundry agent configuration
foundry_agent_config = AgentConfig(
    name="workshop_foundry_agent",
    agent_type="azure_foundry",
    enabled=True,
    instructions="""You are an advanced AI agent powered by Azure AI Foundry, designed for enterprise-grade applications.

ENTERPRISE CAPABILITIES:
- Advanced reasoning and problem-solving
- Integration with Azure ecosystem
- Built-in security and compliance
- Production-ready scalability
- Comprehensive monitoring and analytics

WORKSHOP ROLE:
- Demonstrate enterprise AI capabilities
- Explain Azure AI Foundry benefits
- Provide production-ready examples
- Show integration possibilities

RESPONSE STYLE:
- Professional yet approachable
- Include technical details when relevant
- Highlight enterprise features
- Provide actionable insights
- Use examples from real-world scenarios""",
    metadata={
        "description": "Production-ready Azure AI Foundry agent",
        "capabilities": [
            "enterprise_reasoning",
            "azure_integration", 
            "security_compliance",
            "production_monitoring",
            "advanced_analytics",
            "scalable_deployment"
        ],
        "workshop_level": "advanced",
        "environment": "azure_foundry"
    },
    framework_config={
        "provider": "azure_foundry",
        "model": "gpt-4o",
        "temperature": 0.6,  # Balanced for enterprise use
        "max_tokens": 1200,  # Detailed enterprise responses
        "endpoint": os.getenv("PROJECT_ENDPOINT"),
        "use_managed_identity": True,  # Enterprise security
        "enable_monitoring": True,     # Production monitoring
        "enable_analytics": True       # Usage analytics
    }
)

print("🏢 Azure AI Foundry Agent Configuration Created!")
print("🔑 Key enterprise features:")
print(f"  ✅ Managed Identity: {foundry_agent_config.framework_config.get('use_managed_identity')}")
print(f"  ✅ Monitoring: {foundry_agent_config.framework_config.get('enable_monitoring')}")
print(f"  ✅ Analytics: {foundry_agent_config.framework_config.get('enable_analytics')}")
print(f"  ✅ Provider: {foundry_agent_config.framework_config.get('provider')}")
print(f"  ✅ Endpoint: {foundry_agent_config.framework_config.get('endpoint')}")

In [ ]:
# Create and initialize the Azure AI Foundry agent
try:
    # Use our LangChain Azure Foundry agent implementation
    foundry_agent = LangChainAzureFoundryAgent(foundry_agent_config)
    await foundry_agent.initialize()
    
    print("🚀 Azure AI Foundry Agent Created Successfully!")
    
    # Register in our agent registry
    agent_registry.register_agent("foundry", foundry_agent)
    
    print(f"📋 Agent Registry now contains: {agent_registry.get_all_agents()}")
    print(f"🎯 Foundry agent capabilities: {foundry_agent.get_capabilities()}")
    
    # Show enterprise features
    print("\n🏢 Enterprise Features Enabled:")
    print("  ✅ Secure authentication with managed identity")
    print("  ✅ Built-in request/response monitoring")
    print("  ✅ Automatic retry logic with exponential backoff")
    print("  ✅ Integration with Azure security services")
    print("  ✅ Compliance and governance features")
    print("  ✅ Production-ready scalability")
    
except Exception as e:
    print(f"❌ Error creating Azure AI Foundry agent: {e}")
    print("This might happen if Azure AI Foundry isn't fully configured")
    print("But we can still demonstrate the configuration approach!")

In [ ]:
# Test the Azure AI Foundry agent
print("🧪 Testing Azure AI Foundry Agent...")

if 'foundry_agent' in locals():
    # Test enterprise features
    enterprise_tests = [
        "What are the key benefits of using Azure AI Foundry for enterprise AI applications?",
        "How does managed identity improve security in AI applications?",
        "Can you explain the monitoring and analytics capabilities you provide?",
        "What makes you different from the basic agents we created earlier?"
    ]
    
    for i, test_message in enumerate(enterprise_tests, 1):
        print(f"\n🔬 Test {i}/{len(enterprise_tests)}")
        try:
            response = await foundry_agent.process_message(test_message, [], {
                "test_id": f"enterprise_test_{i}",
                "workshop_session": "langchain_foundry"
            })
            
            print(f"💬 Question: {test_message}")
            print(f"🏢 Foundry Agent: {response.content}")
            print(f"📊 Enterprise Metadata: {response.metadata}")
            print("-" * 80)
            
        except Exception as e:
            print(f"❌ Test {i} failed: {e}")
            
else:
    print("⚠️ Foundry agent not available for testing")
    print("In a real environment, this would demonstrate:")
    print("  - Advanced reasoning capabilities")
    print("  - Enterprise security features")
    print("  - Built-in monitoring and analytics")
    print("  - Production-ready performance")

## Section 7: Compare Agent Performances

🏆 **Time for the Grand Comparison!** Let's compare all the agents we've built and see how they perform on the same tasks.

This section will help you understand:
- The evolution from basic to enterprise agents
- Performance differences between implementations  
- When to use each type of agent
- Real-world application scenarios

In [ ]:
# Comprehensive agent comparison
import time
from typing import Dict, List, Tuple

async def compare_agents(test_message: str, agents_dict: Dict[str, IAgent]) -> Dict[str, Dict]:
    """Compare multiple agents on the same task."""
    results = {}
    
    print(f"🔬 Testing all agents with: '{test_message}'")
    print("=" * 80)
    
    for agent_name, agent in agents_dict.items():
        try:
            start_time = time.time()
            
            # Test the agent
            response = await agent.process_message(test_message, [], {
                "comparison_test": True,
                "agent_name": agent_name
            })
            
            end_time = time.time()
            response_time = round((end_time - start_time) * 1000, 2)  # Convert to milliseconds
            
            # Store results
            results[agent_name] = {
                "response": response.content,
                "response_time_ms": response_time,
                "capabilities": agent.get_capabilities(),
                "metadata": response.metadata,
                "success": True
            }
            
            # Display results
            print(f"🤖 {agent_name.upper()} AGENT:")
            print(f"   Response Time: {response_time}ms")
            print(f"   Response: {response.content[:150]}{'...' if len(response.content) > 150 else ''}")
            print(f"   Capabilities: {agent.get_capabilities()}")
            print("-" * 60)
            
        except Exception as e:
            results[agent_name] = {
                "error": str(e),
                "success": False,
                "response_time_ms": 0
            }
            print(f"❌ {agent_name.upper()} AGENT: Error - {e}")
            print("-" * 60)
    
    return results

# Prepare agents for comparison
agents_to_compare = {}

# Add available agents
if 'basic_agent' in locals():
    agents_to_compare["basic"] = basic_agent
    
if 'enhanced_agent' in locals():
    agents_to_compare["enhanced"] = enhanced_agent
    
if 'foundry_agent' in locals():
    agents_to_compare["foundry"] = foundry_agent

print(f"🎯 Comparing {len(agents_to_compare)} agents:")
for name in agents_to_compare.keys():
    print(f"   ✅ {name.title()} Agent")

In [ ]:
# Test 1: Basic conversation
print("🧪 TEST 1: Basic Conversation")
results_1 = await compare_agents(
    "Hello! Can you explain what makes a good AI agent?", 
    agents_to_compare
)

print("\n" + "="*80 + "\n")

# Test 2: Technical explanation
print("🧪 TEST 2: Technical Explanation")
results_2 = await compare_agents(
    "Explain the benefits of using dependency injection in software architecture.", 
    agents_to_compare
)

print("\n" + "="*80 + "\n")

# Test 3: Enterprise scenario
print("🧪 TEST 3: Enterprise Scenario")
results_3 = await compare_agents(
    "How would you design a scalable AI system for a large enterprise with security and compliance requirements?", 
    agents_to_compare
)

In [ ]:
# Performance analysis
print("📊 PERFORMANCE ANALYSIS")
print("="*80)

def analyze_results(test_name: str, results: Dict):
    """Analyze and display test results."""
    print(f"\n📈 {test_name} Analysis:")
    
    successful_agents = {k: v for k, v in results.items() if v.get('success', False)}
    
    if successful_agents:
        # Response time analysis
        avg_response_time = sum(v['response_time_ms'] for v in successful_agents.values()) / len(successful_agents)
        fastest_agent = min(successful_agents.items(), key=lambda x: x[1]['response_time_ms'])
        
        print(f"   ⚡ Average response time: {avg_response_time:.2f}ms")
        print(f"   🏃 Fastest agent: {fastest_agent[0]} ({fastest_agent[1]['response_time_ms']}ms)")
        
        # Capability analysis
        all_capabilities = set()
        for agent_data in successful_agents.values():
            all_capabilities.update(agent_data.get('capabilities', []))
        
        print(f"   🎯 Total unique capabilities: {len(all_capabilities)}")
        print(f"   📋 Capabilities: {', '.join(sorted(all_capabilities))}")
        
        # Response quality (length as a proxy)
        response_lengths = {k: len(v['response']) for k, v in successful_agents.items()}
        most_detailed = max(response_lengths.items(), key=lambda x: x[1])
        
        print(f"   📝 Most detailed response: {most_detailed[0]} ({most_detailed[1]} characters)")
    
    else:
        print("   ❌ No successful responses for this test")

# Analyze all tests
if 'results_1' in locals():
    analyze_results("Basic Conversation", results_1)
    
if 'results_2' in locals():
    analyze_results("Technical Explanation", results_2)
    
if 'results_3' in locals():
    analyze_results("Enterprise Scenario", results_3)

## 🎉 Congratulations! Workshop Complete!

You've successfully completed the LangChain Agents Workshop! Here's what you've accomplished:

### ✅ **What You've Built:**
1. **Basic Generic Agent** - Simple conversational AI
2. **Enhanced Agent** - With memory and advanced capabilities  
3. **Azure AI Foundry Agent** - Enterprise-ready with security and monitoring

### 🎯 **Key Learnings:**
- **Modern Architecture**: Plugin-based, extensible design
- **Configuration-Driven**: Easy to modify and deploy
- **Security Best Practices**: Using managed identity and secure connections
- **Enterprise Features**: Monitoring, analytics, and scalability

### 🚀 **Next Steps:**
1. **Experiment**: Try different configurations and instructions
2. **Extend**: Add custom tools and capabilities to your agents
3. **Deploy**: Use Azure AI Foundry for production deployment
4. **Monitor**: Implement logging and analytics for your agents

### 📚 **Resources:**
- [Azure AI Foundry Documentation](https://docs.microsoft.com/azure/ai-foundry/)
- [LangChain Documentation](https://python.langchain.com/)
- [Modern Agent Architecture Guide](../../README.md)
- [Configuration Examples](../../examples/)

### 🤝 **Questions & Discussion:**
What questions do you have about building and deploying AI agents?

**Thank you for participating in this workshop!** 🎊

## Section 7: LangChain vs Semantic Kernel - Framework Comparison

Now that you've experienced both workshops, let's compare the frameworks to help you choose the right one for your projects.

In [ ]:
def create_framework_comparison():
    """
    Create a comprehensive comparison between LangChain and Semantic Kernel.
    """
    
    print("🆚 LANGCHAIN vs SEMANTIC KERNEL COMPARISON")
    print("=" * 50)
    
    # Framework comparison matrix
    comparison_data = {
        "Aspect": [
            "Architecture", "Learning Curve", "Tool Ecosystem", "Memory Management",
            "Multi-Provider Support", "Enterprise Features", "Community Size",
            "Microsoft Integration", "Flexibility", "Performance", 
            "Documentation", "Production Readiness"
        ],
        "LangChain": [
            "Chain-based", "Moderate", "Extensive", "Advanced",
            "Excellent", "Good", "Large",
            "Good", "Very High", "Good",
            "Excellent", "Mature"
        ],
        "Semantic Kernel": [
            "Plugin-based", "Easy", "Growing", "Basic",
            "Good", "Excellent", "Medium",
            "Native", "High", "Optimized",
            "Good", "Enterprise-Ready"
        ]
    }
    
    print("\\n📊 DETAILED COMPARISON")
    print("-" * 25)
    
    # Print comparison table
    col_widths = [20, 15, 18]
    headers = ["Aspect", "LangChain", "Semantic Kernel"]
    
    # Print header
    header_row = ""
    for i, header in enumerate(headers):
        header_row += f"{header:<{col_widths[i]}}"
    print(header_row)
    print("-" * sum(col_widths))
    
    # Print rows
    for i, aspect in enumerate(comparison_data["Aspect"]):
        row = f"{aspect:<{col_widths[0]}}"
        row += f"{comparison_data['LangChain'][i]:<{col_widths[1]}}"
        row += f"{comparison_data['Semantic Kernel'][i]:<{col_widths[2]}}"
        print(row)
    
    print("\\n🎯 WHEN TO CHOOSE LANGCHAIN")
    print("-" * 30)
    
    langchain_use_cases = [
        "🔗 Complex chain orchestration and workflows",
        "🛠️ Need extensive pre-built tool integrations",
        "🧠 Advanced memory and retrieval requirements",
        "🌐 Multi-provider flexibility is critical",
        "📚 Rich documentation and community support needed",
        "🔄 Rapid prototyping with diverse components",
        "🐍 Python-first development approach"
    ]
    
    for use_case in langchain_use_cases:
        print(f"   {use_case}")
    
    print("\\n🎯 WHEN TO CHOOSE SEMANTIC KERNEL")
    print("-" * 35)
    
    sk_use_cases = [
        "🏢 Enterprise Microsoft environment",
        "🚀 Quick start with minimal learning curve",
        "🔌 Plugin-based extensibility preferred",
        "⚡ Performance optimization important",
        "🛡️ Enterprise security and compliance focus",
        "🔗 Native Azure integration required",
        "🎯 Simpler, more focused agent requirements"
    ]
    
    for use_case in sk_use_cases:
        print(f"   {use_case}")
    
    print("\\n🤝 HYBRID APPROACH")
    print("-" * 17)
    print("🔄 You can use both frameworks in the same project!")
    print("   • LangChain for complex workflows and tools")
    print("   • Semantic Kernel for Microsoft-integrated components")
    print("   • Choose based on specific use case requirements")
    
    print("\\n🎓 LEARNING RECOMMENDATION")
    print("-" * 25)
    print("📚 Start with: Semantic Kernel (easier learning curve)")
    print("🔄 Then explore: LangChain (for advanced capabilities)")
    print("🎯 Choose based on: Your specific project needs")
    print("💡 Remember: Both are excellent frameworks!")
    
    return comparison_data

# Generate the comparison
comparison_results = create_framework_comparison()

print("\\n✨ Framework comparison complete!")
print("🎯 Now you can make informed decisions about which framework to use!")
print("🚀 Both workshops completed - you're ready for production AI agents!")

## Optional utilities

Use these helpers if you need to:
- Reset or clean up your environment variables and optional .env file
- Start the LangChain FastAPI server (uvicorn) from the notebook
- Stop the server safely on Windows

These are optional and independent from the Quick Start steps above. If you don't need them, you can ignore this section.

In [ ]:
# Utility: Reset config (.env and in-memory)
import os, json, shutil
from pathlib import Path

# Toggle deletion of .env file created in Step 2
DELETE_ENV_FILE = False  # set True to remove .env

# Env vars used by this LangChain app
_ENV_KEYS = [
    "AZURE_INFERENCE_ENDPOINT","AZURE_INFERENCE_CREDENTIAL","GENERIC_MODEL",
    "PROJECT_ENDPOINT","PEOPLE_AGENT_ID","KNOWLEDGE_AGENT_ID",
    "FRONTEND_URL","LOG_LEVEL","ENVIRONMENT",
    "SESSION_STORAGE_TYPE","SESSION_STORAGE_PATH","REDIS_URL",
    "DEBUG_LOGS","CONFIG_PATH"
 ]

def _mask(v):
    if v is None:
        return ""
    return v[:4] + "***" if len(v) > 8 else "***"

def reset_config(delete_env: bool = False):
    # Clear in-memory env
    cleared = {}
    for k in _ENV_KEYS:
        if k in os.environ:
            cleared[k] = os.environ.pop(k)
    # Optionally delete .env in this folder
    env_path = Path(".env")
    removed_env_file = False
    if delete_env and env_path.exists():
        try:
            env_path.unlink()
            removed_env_file = True
        except Exception as e:
            print(f"Warning: couldn't delete .env: {e}")
    print("Cleared env keys:")
    for k, v in cleared.items():
        print(f"- {k} = { _mask(v) }")
    print(f"Removed .env file: {removed_env_file}")
    return {"cleared": list(cleared.keys()), "removed_env_file": removed_env_file}

result = reset_config(DELETE_ENV_FILE)
print("Reset complete.")

In [ ]:
# Utility: Start API server (uvicorn) in background
import os, sys, subprocess, time, json
from pathlib import Path

# Settings
HOST = os.getenv("HOST", "127.0.0.1")
PORT = int(os.getenv("PORT", "8001"))  # avoid conflict with SK if it's 8000
RELOAD = False  # set True during local dev
LOG_LEVEL = os.getenv("LOG_LEVEL", "info")
PID_FILE = Path(".uvicorn_pid")

def start_server():
    if PID_FILE.exists():
        print("A server appears to be running already (PID file exists). If it's stale, run the Stop cell first.")
        return {"status": "skipped", "reason": "pid_exists"}
    cmd = [sys.executable, "-m", "uvicorn", "main:app", "--host", HOST, "--port", str(PORT), "--log-level", LOG_LEVEL]
    if RELOAD:
        cmd.append("--reload")
    # On Windows, creationflags=CREATE_NEW_PROCESS_GROUP helps Ctrl+C and termination
    creationflags = 0x00000200  # CREATE_NEW_PROCESS_GROUP
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, creationflags=creationflags)
    PID_FILE.write_text(str(proc.pid))
    print(f"Starting uvicorn main:app at http://{HOST}:{PORT} (pid={proc.pid})...")
    # Brief wait to give server time to bind
    time.sleep(1.5)
    return {"status": "started", "pid": proc.pid, "url": f"http://{HOST}:{PORT}"}

result = start_server()
result

In [ ]:
# Utility: Stop API server (read PID file and terminate)
import os, signal
from pathlib import Path

PID_FILE = Path(".uvicorn_pid")

def stop_server():
    if not PID_FILE.exists():
        print("No PID file found. If a server is running, you may need to stop it manually.")
        return {"status": "skipped", "reason": "no_pid"}
    try:
        pid = int(PID_FILE.read_text().strip())
    except Exception as e:
        print(f"Couldn't read pid: {e}")
        return {"status": "error", "error": str(e)}
    try:
        # Windows-friendly termination: first try CTRL_BREAK_EVENT, then terminate
        try:
            os.kill(pid, signal.CTRL_BREAK_EVENT)
        except Exception:
            # Fallback to terminate
            os.kill(pid, signal.SIGTERM)
        print(f"Sent termination to pid {pid}")
    except Exception as e:
        print(f"Error signaling process: {e}")
        return {"status": "error", "error": str(e)}
    try:
        PID_FILE.unlink()
    except Exception:
        pass
    return {"status": "stopped", "pid": pid}

result = stop_server()
result